In [32]:
import pandas as pd
import numpy as np
import gmaps
import requests
from citipy import citipy
from config import g_key
from datetime import datetime
import time
from scipy.stats import linregress

In [20]:
city_data=pd.read_csv('challenge_weather_data/WeatherPy_challenge.csv')
city_data.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Rikitea,PF,2020-01-04 20:04:54,-23.12,-134.97,77.20,77,67,16.60,broken clouds,0.0,0.0
1,1,Haines Junction,CA,2020-01-04 20:03:47,60.75,-137.51,-8.25,97,100,3.31,overcast clouds,0.0,0.0
2,2,Pedasi,PA,2020-01-04 20:04:54,7.53,-80.03,83.05,71,63,6.22,broken clouds,0.0,0.0
3,3,Dikson,RU,2020-01-04 20:04:55,73.51,80.55,0.25,82,100,22.15,overcast clouds,0.0,0.0
4,4,Bemmel,NL,2020-01-04 20:08:47,51.89,5.90,45.00,100,75,6.93,heavy intensity drizzle,0.0,0.0


In [34]:
gmaps.configure(api_key=g_key)

In [25]:

min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input('What is the maximum temperature you would like for your trip? '))
raining=input('Do you want it to be raining? (yes/no) ')
snowing=input('Do you want it to be snowing? (yes/no) ')

preferred_cities=city_data.loc[(city_data['Max Temp']<=max_temp)& \
                               (city_data['Max Temp']>= min_temp)]

# 4 different Combos:
if raining == 'no' and snowing == 'no':
    preferred_cities=preferred_cities.loc[(preferred_cities['Snow inches (last 3 hrs)']==0)&  
                                          (preferred_cities['Rain inches (last 3 hrs)']==0)]
    
elif raining == 'no' and snowing == 'yes':
    preferred_citie=preferred_cities.loc[(preferred_cities['Snow inches (last 3 hrs)']>0)&  
                                          (preferred_cities['Rain inches (last 3 hrs)']==0)]
    
elif raining == 'yes' and snowing == 'no':
    preferred_cities=preferred_cities.loc[(preferred_cities['Snow inches (last 3 hrs)']==0)&  
                                          (preferred_cities['Rain inches (last 3 hrs)']>0)]

else:
    preferred_cities=preferred_cities.loc[(preferred_cities['Snow inches (last 3 hrs)']>0)&  
                                          (preferred_cities['Rain inches (last 3 hrs)']>0)]
preferred_cities.head(10)  

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Rikitea,PF,2020-01-04 20:04:54,-23.12,-134.97,77.20,77,67,16.60,broken clouds,0.0,0.0
2,2,Pedasi,PA,2020-01-04 20:04:54,7.53,-80.03,83.05,71,63,6.22,broken clouds,0.0,0.0
6,6,Arraial Do Cabo,BR,2020-01-04 20:08:48,-22.97,-42.02,77.00,94,75,4.70,light rain,0.0,0.0
10,10,Port-Gentil,GA,2020-01-04 20:08:49,-0.72,8.78,78.80,94,90,6.93,thunderstorm with light rain,0.0,0.0
12,12,Nha Trang,VN,2020-01-04 20:04:58,12.24,109.19,77.00,69,40,8.05,scattered clouds,0.0,0.0
14,14,Georgetown,GY,2020-01-04 20:01:17,6.80,-58.16,86.00,66,20,9.17,few clouds,0.0,0.0
17,17,San Vicente De Canete,PE,2020-01-04 20:04:59,-13.08,-76.38,72.41,84,84,9.15,broken clouds,0.0,0.0
19,19,Salalah,OM,2020-01-04 20:03:33,17.01,54.10,77.00,73,20,10.29,few clouds,0.0,0.0
22,22,Fare,PF,2020-01-04 20:05:00,-16.70,-151.02,81.61,76,100,11.63,overcast clouds,0.0,0.0
25,25,Atuona,PF,2020-01-04 20:05:01,-9.80,-139.03,81.55,68,5,9.24,clear sky,0.0,0.0


In [39]:
hotel_vacation=preferred_cities[['City','Country','Max Temp','Lat','Lng','Current Description']].copy()
hotel_vacation['Hotel Name']=''
hotel_vacation

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Rikitea,PF,77.20,-23.12,-134.97,broken clouds,
2,Pedasi,PA,83.05,7.53,-80.03,broken clouds,
6,Arraial Do Cabo,BR,77.00,-22.97,-42.02,light rain,
10,Port-Gentil,GA,78.80,-0.72,8.78,thunderstorm with light rain,
12,Nha Trang,VN,77.00,12.24,109.19,scattered clouds,
...,...,...,...,...,...,...,...
526,Monteagudo,BO,89.60,-19.80,-63.96,scattered clouds,
538,Mitsamiouli,KM,81.81,-11.38,43.28,overcast clouds,
539,La Rioja,AR,83.44,-29.41,-66.86,clear sky,
541,Tomatlan,MX,86.68,19.93,-105.25,scattered clouds,


In [40]:
params={
    'radius':5000,
    'types':'lodging',
    'key':g_key}
base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
hotels = requests.get(base_url, params=params).json()

In [41]:
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f'{lat},{lng}'
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_vacation.loc[index,"Hotel Name"]=hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [42]:
hotel_vacation

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Rikitea,PF,77.20,-23.12,-134.97,broken clouds,Pension Maro'i
2,Pedasi,PA,83.05,7.53,-80.03,broken clouds,Selina Pedasi
6,Arraial Do Cabo,BR,77.00,-22.97,-42.02,light rain,Pousada Porto Praia
10,Port-Gentil,GA,78.80,-0.72,8.78,thunderstorm with light rain,Fengshui Residence
12,Nha Trang,VN,77.00,12.24,109.19,scattered clouds,Hanoi Golden Hotel
...,...,...,...,...,...,...,...
526,Monteagudo,BO,89.60,-19.80,-63.96,scattered clouds,Hogar dulce hogar
538,Mitsamiouli,KM,81.81,-11.38,43.28,overcast clouds,Foyer ADM
539,La Rioja,AR,83.44,-29.41,-66.86,clear sky,Hotel Libertador
541,Tomatlan,MX,86.68,19.93,-105.25,scattered clouds,Norma franco


In [43]:
output_data_file='challenge_weather_data/WeatherPy_vacation.csv'
hotel_vacation.to_csv(output_data_file, index_label='City_ID')

In [30]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
hotel_info=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [35]:
locations=hotel_df[['Lat','Lng']]
max_temp=hotel_df['Max Temp']
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
conda install -c conda-forge ipywidgets

Solving environment: done

## Package Plan ##

  environment location: /Users/danielchang/anaconda3

  added / updated specs:
    - ipywidgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ipywidgets-7.5.1           |             py_0         101 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         101 KB

The following packages will be SUPERSEDED by a higher-priority channel:

  ipywidgets                                      pkgs/main --> conda-forge



ipywidgets-7.5.1     | 101 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.
